In [1]:
import os,sys,glob
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

Using TensorFlow backend.


In [2]:
model = sent2vec.Sent2vecModel()
model.load_model('model/o2o_o2m_Sent2Vec_lower_woParam_0616_768.bin') # sent2vec model
all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [3]:
# encode_dict = {}
# for i,v in enumerate(api_li):
#     encode_dict[v] = i+1
# pickle.dump(file=open('data/tree-rep-profiles-partial/encode_dict.pkl','wb'),obj=encode_dict)
# encode_dict
encode_dict = pickle.load(open('data/tree-rep-profiles-partial/encode_dict.pkl','rb'))
encode_dict

{'LoadLibrary': 1,
 'CreateProcess': 2,
 'OpenProcess': 3,
 'ExitProcess': 4,
 'TerminateProcess': 5,
 'WinExec': 6,
 'CreateRemoteThread': 7,
 'CreateThread': 8,
 'CopyFile': 9,
 'CreateFile': 10,
 'DeleteFile': 11,
 'RegSetValue': 12,
 'RegCreateKey': 13,
 'RegDeleteKey': 14,
 'RegDeleteValue': 15,
 'RegQueryValue': 16,
 'RegEnumValue': 17,
 'WinHttpConnect': 18,
 'WinHttpOpen': 19,
 'WinHttpOpenRequest': 20,
 'WinHttpReadData': 21,
 'WinHttpSendRequest': 22,
 'InternetOpen': 23,
 'InternetConnect': 24,
 'HttpSendRequest': 25,
 'GetUrlCacheEntryInfo': 26}

In [4]:
# lower = True:

encode_dict_lower = {}
for i,v in encode_dict.items():
    encode_dict_lower[i.lower()] = v
encode_dict_lower

{'loadlibrary': 1,
 'createprocess': 2,
 'openprocess': 3,
 'exitprocess': 4,
 'terminateprocess': 5,
 'winexec': 6,
 'createremotethread': 7,
 'createthread': 8,
 'copyfile': 9,
 'createfile': 10,
 'deletefile': 11,
 'regsetvalue': 12,
 'regcreatekey': 13,
 'regdeletekey': 14,
 'regdeletevalue': 15,
 'regqueryvalue': 16,
 'regenumvalue': 17,
 'winhttpconnect': 18,
 'winhttpopen': 19,
 'winhttpopenrequest': 20,
 'winhttpreaddata': 21,
 'winhttpsendrequest': 22,
 'internetopen': 23,
 'internetconnect': 24,
 'httpsendrequest': 25,
 'geturlcacheentryinfo': 26}

In [5]:
root_dir = './data/tree-rep-profiles-partial/normal/'
av_col = all_df.columns[1:].tolist()
fam_dir = next(os.walk(root_dir))[1]
fam_dir = [x.split('_')[0] for x in fam_dir]
av_df_col = []
for name in av_col:
    if name in fam_dir:
        av_df_col.append(name)
# av_df_col = ['profile'] + av_df_col
print(len(av_df_col))
av_df_col

44


['3.kazy',
 '4.zbot',
 '6.allaple',
 '11.virtob',
 '12.loadmoney',
 '13.graftor',
 '14.vobfus',
 '17.expiro',
 '18.sytro',
 '21.ramnit',
 '22.domaiq',
 '27.softpulse',
 '29.delf',
 '30.mplug',
 '32.browsefox',
 '33.autoit',
 '37.parite',
 '38.msil',
 '41.outbrowse',
 '42.sirefef',
 '43.mira',
 '44.screensaver',
 '45.firseria',
 '47.somoto',
 '51.shodi',
 '54.soltern',
 '55.zygug',
 '59.megasearch',
 '60.conjar',
 '63.jorik',
 '64.installerex',
 '65.shipup',
 '66.bdmj',
 '71.eggnog',
 '75.fesber',
 '78.vilsel',
 '81.picsys',
 '84.directdow',
 '88.clickdownload',
 '103.yantai',
 '104.mresmon',
 '111.vbran',
 '124.avmh',
 '151.downloadadmin']

In [6]:
def create_fam_data(root_dir,samples_pid,av_df_col=av_df_col):
# root_dir = './data/tree-rep-profiles-partial/normal/'
# samples_pid = samples_pid_train
    name_df = pd.DataFrame(0,columns=av_df_col,index=samples_pid)
    for pid in tqdm(samples_pid):
        paths = glob.glob(root_dir +'*/*/'+pid )
        name_li = []
        for path in paths:
            name_li.append(path.split('/')[4].split('_')[0])
        name_df.loc[pid,list(set(name_li))] = 1
    return name_df

In [7]:
samples_pid_train = pickle.load(file=open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(file=open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(file=open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))


In [8]:
# root_dir = './data/tree-rep-profiles-partial/normal/'
# train_fam_df = create_fam_data(root_dir,samples_pid_train)
# root_dir = './data/tree-rep-profiles-partial/DEV/'
# valid_fam_df = create_fam_data(root_dir,samples_pid_valid)
# root_dir = './data/tree-rep-profiles-partial/TEST/'
# test_fam_df = create_fam_data(root_dir,samples_pid_test)
train_fam_df, valid_fam_df,test_fam_df = pickle.load(open('data/tree-rep-profiles-partial/fam_df.pkl','rb'))

100%|██████████| 264/264 [00:00<00:00, 301.65it/s]


In [9]:
def byterep_ans(byterep_list,max_length):
    '''
    Input: byte sequenxe (list of pickles path) 
    '''
    rep_list=[]
    for byterep in byterep_list:
        rep = pickle.load(open(byterep,'rb'))
        if len(rep) < max_length:
            for _ in range(max_length-len(rep)):
                rep.append(0)
        rep_list.append(rep)
    return rep_list

# 寫不同模組 來處理x2 y1 y2
def Sent2vec_emb(root_dir,all_df,max_length,model=model,dim=768,encode_dict=encode_dict,av_df_col=av_df_col):
    '''
    Create sent2vec profile embedding
    Input: root_dir=>fam-tree-profile
    Output: 
    '''
    sent_pad = [0]*dim
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    sen2vec_dim_normalize=[]
    sen2vec_length_normalize = []
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            byterep_list = [x.split('.')[0]+'_byterep.pickle' for x in hl_list]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            byterep_list = [os.path.join(in_directory, f) for f in byterep_list]
            rep_list = byterep_ans(byterep_list,max_length)
            for profile in hl_list:
                pid = profile.split('/')[-1]#.split('.')[0]
#                 profile_df = all_df[all_df.profile == pid]
                profile_df = all_df[all_df.index == pid]
                fam_ans = list(profile_df.values[0])
#                 fam_ans = list(profile_df[av_df_col].values[0])

                profile_emb = []
                func_name_emb = []
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line.lower()) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0]
                    if func_name not in api_li:
                        print('=ERROR:=',profile,'=>',temp)
                    func_name_emb.append(encode_dict[func_name])
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb)
                    sen2vec_dim_normalize.append(emb)
                    sen2vec_length_normalize.append(len(lines))
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                all_profiles.append(profile_emb)
                all_api_name.append(func_name_emb)
                all_fam_ans.append(fam_ans)
            all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0)
    return np.array(all_profiles) , np.array(all_api_name) , np.array(all_fam_ans) , np.array(all_byterep),np.array(sen2vec_dim_normalize),sen2vec_length_normalize

In [10]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"\=\_]+\ *" #等號、底線被保留，注意要跟先前一致
max_length = 213

root_dir = './data/tree-rep-profiles-partial/normal/'
train_emb, train_emb_api, train_fam_ans, train_rep_ans, train_norm,train_norm_len = Sent2vec_emb(root_dir,train_fam_df,max_length,encode_dict=encode_dict_lower)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/DEV/'
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans, valid_norm,valid_norm_len  = Sent2vec_emb(root_dir,valid_fam_df,max_length,encode_dict=encode_dict_lower)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/TEST/'
test_emb, test_emb_api,test_fam_ans,test_rep_ans, test_norm,test_norm_len = Sent2vec_emb(root_dir,test_fam_df,max_length,encode_dict=encode_dict_lower)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

for val in test_emb:
    if len(val) != max_length:
        print(len(val))

  0%|          | 0/44 [00:00<?, ?it/s]

train of sent2vec vector: (11141, 213, 768) (11141, 213) (11141, 44) (11141, 213)


  2%|▏         | 1/44 [00:00<00:06,  7.04it/s]

valid of sent2vec vector: (437, 213, 768) (437, 213) (437, 44) (437, 213)


100%|██████████| 44/44 [00:07<00:00,  4.48it/s]


test of sent2vec vector: (424, 213, 768) (424, 213) (424, 44) (424, 213)


## normalize dim-wise

In [11]:
alls = np.concatenate([test_norm,valid_norm,train_norm])
mean = np.mean(alls,axis=0)
std = np.std(alls,axis=0)
mean.shape , std.shape

((768,), (768,))

In [12]:
def normalize_dim(emb,length,mean=mean,std=std):
    sent_emb = (emb[:length,:] - mean)/std
    final_emb = np.concatenate([sent_emb, emb[length:,:]],axis=0)
    return final_emb

In [13]:
test_emb_norm = []
valid_emb_norm = []
train_emb_norm = []
for emb,length in zip(train_emb,train_norm_len):
    emb_norm = normalize_dim(emb,length)
    train_emb_norm.append(emb_norm)
for emb,length in zip(valid_emb,valid_norm_len):
    emb_norm = normalize_dim(emb,length)
    valid_emb_norm.append(emb_norm)
for emb,length in zip(test_emb,test_norm_len):
    emb_norm = normalize_dim(emb,length)
    test_emb_norm.append(emb_norm)

In [14]:
train_emb = np.array(train_emb_norm)
valid_emb = np.array(valid_emb_norm)
test_emb = np.array(test_emb_norm)
print(train_emb.shape,valid_emb.shape,test_emb.shape)

(11141, 213, 768) (437, 213, 768) (424, 213, 768)


In [15]:
pickle.dump(file=open('data/tree-rep-profiles-partial/TRAIN_sent2vec_vec.pkl','wb'),protocol=4,obj=(train_emb, train_emb_api, train_fam_ans, train_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/DEV_sent2vec_vec.pkl','wb'),protocol=4,obj=(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/TEST_sent2vec_vec.pkl','wb'),protocol=4,obj=(test_emb, test_emb_api,test_fam_ans,test_rep_ans))
# pickle.dump(file=open('data/tree-rep-profiles-partial/fam_df.pkl','wb'),protocol=4,obj=(train_fam_df, valid_fam_df,test_fam_df))
pickle.dump(file=open('data/tree-rep-profiles-partial/norm_mean_std.pkl','wb'),protocol=4,obj=(mean,std))